## Review

Review [Finetune_BERT_for_Sentiment_Classification_maxlength128.ipynb](Finetune_BERT_for_Sentiment_Classification_maxlength128.ipynb) (with `max_length = 128`) to understand the data and the model setup.

Hyperparameter choices in this notebook:
- Batch size (`batch_size`) = 32
- Learning rate (`lr`) = 2e-5
- Number of epochs (`epochs`) = 4
- **Maximum Sequence Length (`max_length`) = 256**

## Results

```
epoch 1: Avg Loss 0.2471, Train Acc 0.8990, Val Acc 0.9232
epoch 2: Avg Loss 0.1423, Train Acc 0.9479, Val Acc 0.9252
epoch 3: Avg Loss 0.0778, Train Acc 0.9738, Val Acc 0.9262
epoch 4: Avg Loss 0.0452, Train Acc 0.9864, Val Acc 0.9256
```

```
Test Acc: 0.9226
```
We observe overfitting with epochs 3 and 4. Thus, we would recommend stopping at epoch 2. Notice that doubling max length resulted in an increase in accuracy by about `2%`. However, training time has increased by about a factor of 2. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer

## Define device (CPU or GPU)
This notebook is run on a MacBook Pro with Apple M2 Max chip. Thus, device is set to "mps" for GPU. For other hardware architecture use:

```Python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
```


In [3]:
device = torch.device("mps")

## Load the dataset of movie reviews

In [4]:
df = pd.read_csv("data/IMDB Dataset.csv")
print(f'Number of examples is: {len(df)}')
df.head()

Number of examples is: 50000


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# df = df.sample(n = 1000, random_state=42)  # run on a small subset of data to test code
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Load BERT tokenizer and model


In [6]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 labels: positive and negative
# The informational message below indicates that 
# the model's output layer, including the logits, is randomly initialized.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Inspect model config
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
# Inspect the maximum sequence length
tokenizer.model_max_length

512

In [9]:
# Define a function to tokenize and encode the text and return both input_ids and attention_mask
def tokenize_and_encode(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        truncation=True,
        padding='max_length',
        max_length=256,
        return_attention_mask=True,  # Return attention_mask
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask

tqdm.pandas()

# Apply the function to the "review" column
df[['input_ids', 'attention_mask']] = df['review'].progress_apply(lambda x: tokenize_and_encode(x)).apply(pd.Series)

# Convert sentiments to boolean values
df["sentiment_bool"] = df["sentiment"].apply(lambda x: 1 if x == "positive" else 0)

df.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [01:53<00:00, 438.64it/s]


,review,sentiment,input_ids,attention_mask,sentiment_bool
0,One of the other reviewers has mentioned that ...,positive,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
1,A wonderful little production. <br /><br />The...,positive,"[101, 1037, 6919, 2210, 2537, 1012, 1026, 7987...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,I thought this was a wonderful way to spend ti...,positive,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
3,Basically there's a family where a little boy ...,negative,"[101, 10468, 2045, 1005, 1055, 1037, 2155, 207...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"[101, 9004, 3334, 4717, 7416, 1005, 1055, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1


## Train / Validate / Test split

In [10]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

## Convert data to PyTorch tensors

In [11]:
def get_tensor_data(data):
    out_input_ids = torch.tensor(data["input_ids"].tolist())
    out_attention_masks = torch.tensor(data["attention_mask"].tolist())
    out_labels = torch.tensor(data["sentiment_bool"].tolist())
    return out_input_ids, out_attention_masks, out_labels

train_input_ids, train_attention_masks, train_labels = get_tensor_data(train_df)
val_input_ids, val_attention_masks, val_labels = get_tensor_data(val_df)
test_input_ids, test_attention_masks, test_labels = get_tensor_data(test_df)

## Create DataLoader for efficient batch processing
When you create a DataLoader with shuffle=True, it shuffles the data once at the beginning when the DataLoader is created for that epoch. So, within a single epoch, the order of batches remains the same, but when you start a new epoch, the data is shuffled again, leading to a different order of examples for each epoch. This is a common practice in training deep learning models to ensure that the model doesn't memorize the order of examples and generalizes well across different batches and epochs.

In [12]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

In [13]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Define optimizer and loss function


In [14]:
# Note: setting output_attentions and output_hidden_states to False can help with efficiency
# because it reduces the amount of additional information that the model needs to compute and
# store during forward passes. 
model = BertForSequenceClassification.from_pretrained(model_name, num_labels = 2, 
                                                      output_attentions = False, 
                                                      output_hidden_states = False)

# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
# weight_decay (float, optional) – weight decay coefficient (default: 1e-2)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5) 
criterion = nn.CrossEntropyLoss() # binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.config.hidden_dropout_prob

0.1

## Model training and validation

In [16]:
# Move model to the device (GPU), which is 'mps' on Mac M2
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
epochs = 4
torch.manual_seed(12345)
for epoch in tqdm(range(epochs)):
    # train
    model.train()
    total_loss = 0.0
    correct = 0
    for batch in train_loader:
        num_batches = len(train_loader)
        inputs, attention_mask, labels = batch
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)
        outputs_dict = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs_dict.logits, labels)
        predicted = outputs_dict.logits.argmax(dim=1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Increment the number of correct predictions
        correct += (predicted == labels).type(torch.float).sum().item()
    train_acc = correct / len(train_loader.dataset)
    avg_train_loss = total_loss / num_batches
    
    # validate
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_mask, labels = batch
            inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
            outputs_dict = model(inputs, attention_mask=attention_mask, labels=labels)
            val_predicted = outputs_dict.logits.argmax(dim=1)
            val_correct += (val_predicted == labels).type(torch.float).sum().item()
    val_acc = val_correct / len(val_loader.dataset)
    
    print(f'epoch {epoch + 1}: Avg Loss {avg_train_loss:.4f}, Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}')          
        

 25%|█████████████████████████                                                                           | 1/4 [22:38<1:07:54, 1358.22s/it]

epoch 1: Avg Loss 0.2471, Train Acc 0.8990, Val Acc 0.9232


 50%|███████████████████████████████████████████████████                                                   | 2/4 [45:10<45:09, 1354.98s/it]

epoch 2: Avg Loss 0.1423, Train Acc 0.9479, Val Acc 0.9252


 75%|███████████████████████████████████████████████████████████████████████████                         | 3/4 [1:08:07<22:44, 1364.71s/it]

epoch 3: Avg Loss 0.0778, Train Acc 0.9738, Val Acc 0.9262


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [1:30:56<00:00, 1364.23s/it]

epoch 4: Avg Loss 0.0452, Train Acc 0.9864, Val Acc 0.9256


In [18]:
model.eval()
test_correct = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, attention_mask, labels = batch
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)
        outputs_dict = model(inputs, attention_mask=attention_mask, labels=labels)
        test_predicted = outputs_dict.logits.argmax(dim=1)
        test_correct += (test_predicted == labels).type(torch.float).sum().item()
test_acc = test_correct / len(test_loader.dataset)

print(f'epoch {epoch + 1}: Avg Loss {avg_train_loss:.4f}, Train Acc {train_acc:.4f}, Val Acc {val_acc:.4f}, Test Acc: {test_acc:.4f}')          


epoch 4: Avg Loss 0.0452, Train Acc 0.9864, Val Acc 0.9256, Test Acc: 0.9226


In [19]:
# number of model parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of model parameters: {num_params}")

Number of model parameters: 109483778
